In [112]:
import pandas as pd
import warnings
warnings.filterwarnings('ignore')

In [113]:
df = pd.read_csv('InputData.csv')

In [114]:
df.head()


,name,age,lifestyle,zip code,family status,car,sports,earnings,label,Living area
0,VnSEFOuL,62.0,cozily,50168.0,married,practical,athletics,102526.0,no response,urban
1,8Tv0hcce,34.0,active,66479.0,married,expensive,soccer,33006.0,no response,urban
2,Zny9ysbk,69.0,healthy,16592.0,single,expensive,badminton,118760.0,response,urban
3,HV3xCamM,57.0,cozily,50068.0,married,practical,soccer,131429.0,response,urban
4,sflRsQ6v,66.0,cozily,35988.0,single,practical,badminton,96003.0,response,urban


In [115]:
df['lifestyle'] = pd.factorize(df['lifestyle'])[0]
df['family status'] = pd.factorize(df['family status'])[0]
df['car'] = pd.factorize(df['car'])[0]
df['sports'] = pd.factorize(df['sports'])[0]
df['Living area'] = pd.factorize(df['Living area'])[0]
df['label'] = pd.factorize(df['label'])[0]

In [116]:
df.head()

,name,age,lifestyle,zip code,family status,car,sports,earnings,label,Living area
0,VnSEFOuL,62.0,0,50168.0,0,0,0,102526.0,0,0
1,8Tv0hcce,34.0,1,66479.0,0,1,1,33006.0,0,0
2,Zny9ysbk,69.0,2,16592.0,1,1,2,118760.0,1,0
3,HV3xCamM,57.0,0,50068.0,0,0,1,131429.0,1,0
4,sflRsQ6v,66.0,0,35988.0,1,0,2,96003.0,1,0


In [117]:
y = df['label']
df.drop(['name', 'label'], axis=1, inplace=True)

In [118]:
from sklearn.model_selection import train_test_split, StratifiedKFold
from sklearn.neighbors import KNeighborsClassifier
from sklearn.preprocessing import StandardScaler
from sklearn.tree import DecisionTreeClassifier

X_train, X_holdout, y_train, y_holdout = train_test_split(df.values, y, test_size=0.3,
random_state=17)

tree = DecisionTreeClassifier(max_depth=3, max_features=5, random_state=6)
tree.fit(X_train, y_train)



DecisionTreeClassifier(class_weight=None, criterion='gini', max_depth=3,
            max_features=5, max_leaf_nodes=None, min_impurity_decrease=0.0,
            min_impurity_split=None, min_samples_leaf=1,
            min_samples_split=2, min_weight_fraction_leaf=0.0,
            presort=False, random_state=6, splitter='best')

In [119]:
from sklearn.metrics import accuracy_score

tree_pred = tree.predict(X_holdout)
accuracy_score(y_holdout, tree_pred) 

0.9193333333333333

In [120]:
from sklearn.model_selection import GridSearchCV, cross_val_score

tree_params = {'max_depth': range(1,11),
               'max_features': range(4,8)}

tree_grid = GridSearchCV(tree, tree_params,
                         cv=5, n_jobs=-1, verbose=True)

tree_grid.fit(X_train, y_train)


Fitting 5 folds for each of 40 candidates, totalling 200 fits


[Parallel(n_jobs=-1)]: Using backend LokyBackend with 4 concurrent workers.
[Parallel(n_jobs=-1)]: Done 200 out of 200 | elapsed:    0.9s finished


GridSearchCV(cv=5, error_score='raise-deprecating',
       estimator=DecisionTreeClassifier(class_weight=None, criterion='gini', max_depth=3,
            max_features=5, max_leaf_nodes=None, min_impurity_decrease=0.0,
            min_impurity_split=None, min_samples_leaf=1,
            min_samples_split=2, min_weight_fraction_leaf=0.0,
            presort=False, random_state=6, splitter='best'),
       fit_params=None, iid='warn', n_jobs=-1,
       param_grid={'max_depth': range(1, 11), 'max_features': range(4, 8)},
       pre_dispatch='2*n_jobs', refit=True, return_train_score='warn',
       scoring=None, verbose=True)

In [121]:
tree_grid.best_params_

{'max_depth': 3, 'max_features': 5}

In [122]:
tree_grid.best_score_ 

0.9211428571428572

In [123]:
accuracy_score(y_holdout, tree_grid.predict(X_holdout)) 

0.9193333333333333

In [124]:
import pydotplus #pip install pydotplus
from sklearn.tree import export_graphviz

def tree_graph_to_png(tree, feature_names, png_file_to_save, class_dict):
    tree_str = export_graphviz(tree, feature_names=feature_names, 
                                     filled=True, out_file=None, class_names=class_dict)
#     print(tree_str)
    graph = pydotplus.graph_from_dot_data(tree_str)  
    graph.write_png(png_file_to_save)

In [127]:
tree_graph_to_png(tree=tree_grid.best_estimator_, feature_names=df.columns,
                 png_file_to_save='altran_tree.png', class_dict={0:'no', 1:'yes'})